In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold 
from sklearn.linear_model import LogisticRegression
SEED = 2018

In [ ]:
def get_oof(clf,x_train,y_train,x_test):
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]
    kf = KFold(n_splits=5,random_state=SEED)
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((5,ntest))
    for i,(train_index,test_index) in enumerate(kf.split(x_train)):
        kf_x_train = x_train[train_index]
        kf_y_train = y_train[train_index]
        kf_x_test = x_train[test_index]
        clf.train(kf_x_train,kf_y_train)
        oof_train[test_index] = clf.predict(kf_x_test)
        oof_test_skf[i,:] = clf.predict(x_test)
        
    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1,1),oof_test.reshape(-1,1)

In [ ]:
def model_stacking(clf_set,x_train,y_train,x_test):
    train_stack = np.empty((ntrain,len(clf_set)))
    test_stack = np.empty((ntest,len(clf_set)))
    i=0
    for clf in clf_set:
        oof_train,oof_test = get_oof(clf,x_train,y_train,x_test)
        train_stack[:,i] = oof_train
        test_stack[:,i] = oof_test
        i+=1
    lr = LogisticRegression()
    lr.fit(train_stack,y_train)
    y_pre = lr.predict(test_stack)
    return y_pre

In [1]:
def f_print(a):
    print(a)